In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [152]:
df = pd.read_csv('../notebooks/data.csv')

In [153]:
df = df.drop(columns=['Unnamed: 0'])
df.head(3)

,id,rooms,metro,area,floor,parking,price,renovation,room_area,balcony,windows,bathroom,allowed_children_pets,ceiling_height,city
0,273614615,2,Арбатская,58.0,12,unknown,225000.0,Евроремонт,19.000000,unknown,На улицу и двор,Совмещенный (2),unknown,3.9,Москва
1,274475342,3,Смоленская,98.0,2,подземная,250000.0,Евроремонт,21.000000,unknown,Во двор,"Совмещенный (1), Раздельный (1)","Можно с детьми, Можно с животными",3.2,Москва
2,273973191,3,Смоленская,120.0,5,открытая,130000.0,Евроремонт,31.666667,unknown,На улицу,Совмещенный (1),Можно с животными,3.0,Москва


In [ ]:
# делим на численные и категориальные
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop('price') 
categorical_cols = df.select_dtypes(include=['object']).columns
# кодируем категориальные
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols))
# кодируем численные
scaler = StandardScaler()
scaled = scaler.fit_transform(df[numeric_cols])
scaled_df = pd.DataFrame(scaled, columns=numeric_cols)
# объединяем в один датафрейм
encoded_df_final = pd.concat([encoded_df, scaled_df], axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd
# Целевая переменная
y = df['price']
# Делим выборку
X_train, X_test, y_train, y_test = train_test_split(encoded_df_final, y, test_size=0.2, random_state=42)
# Обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)
# Предсказания
y_pred = model.predict(X_test)
# Считаем MAPE
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print(f"MAPE: {mape:.2f}%")


MAPE: 20.89%


In [ ]:
# другой способ кодировки
encoded_df_final_2 = pd.get_dummies(df)
# Кодируем все категориальные переменные с помощью get_dummies
encoded_df_final_2 = pd.get_dummies(df.drop(columns=['id']))  # убираем id, он не нужен
# Целевая переменная
y = df['price']
# Признаки (после кодировки)
X = encoded_df_final_2.drop(columns=['price'])
# Делим на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)
# Делаем предсказания
y_pred = model.predict(X_test)
# Считаем MAPE
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print(f"MAPE: {mape:.2f}%")


MAPE: 20.94%
